In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import PIL
import PIL.Image
import pandas as pd

import pathlib
import os

img_height = 32
img_width = 32
batch_size = 32
split_seed = 41123

In [ ]:
directory = "data/"
df = pd.read_csv(directory + "data.csv")

file_paths = df["file_name"].values
labelx = df["labelx"].values
labely = df["labely"].values

ds_train = tf.data.Dataset.from_tensor_slices((file_paths, labelx, labely))
image_paths = df["file_name"].values
coordinates = pd.read_csv("data/data.csv")

def load_image(image_path):
    image = tf.io.read_file(directory + image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [img_height, img_width])
    image = (255.0 - image) / 255.0
    return image

def load_data(image_path, label):
    image = load_image(image_path)
    return image, label

image_paths_tensor = tf.constant(image_paths)
coordinates_tensor = tf.convert_to_tensor(coordinates[["labelx", "labely"]].values, dtype=tf.float32)

dataset = tf.data.Dataset.from_tensor_slices((image_paths_tensor, coordinates_tensor))
dataset = dataset.map(lambda path, coords: (load_image(path), coords))
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=32, seed=split_seed)

# Train-test split
train_ratio = 0.7
total_size = 1438
train_size = int(total_size * train_ratio)
test_size = total_size - train_size

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(42, activation='relu'))
model.add(layers.Dense(2, activation='relu'))

model.compile(
    optimizer="adam",
    loss="mean_squared_error"
)

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [ ]:
loss_history = model.history.history["loss"]
print(min(loss_history))

plt.figure()
plt.plot(loss_history)

In [ ]:
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [32, 32]) 
    image = (255.0 - image) / 255.0
    return tf.expand_dims(image, axis=0)
test_image = preprocess_image("img_test.png")
model(test_image)

In [ ]:
model.export("model")